In [ ]:
from pathlib import Path

import pandas as pd
import nltk

import sys
sys.path.append('..')
from src.table import Table, Key
from src.data.loader import ExcelLoader
from src.code.wals import language_to_wals_code

# Building table data structure

## Dataset path

In [ ]:
multilingual = Path('../data/aligned_verses_spreadsheets/')
greek = Path('../data/Greek')

### ExcelLoader

In [ ]:
excelloader = ExcelLoader(multilingual, greek)

## Table

In [ ]:
table = Table(excelloader, language_to_wals_code)

In [ ]:
for k in table(lang=['GREEK'], chap='LUK.19'):
    print(k)
    break

In [ ]:
len(table._data)

# Queries

In [ ]:
for p in iter_fixed(data, book='MRK', chap='MRK.1', lang='ENGLISH', vers=0):
    print(p)

### Language chapters

In [ ]:
def language_chapters(data):
    lang_ch = defaultdict(list)
    for e, v in iter_fixed(data):
        lang_ch[e.lang].append((e.chap, e.vers))
    return lang_ch

In [ ]:
%%timeit
lc = language_chapters(data)

In [ ]:
print(len(lc['GERMANY'])) # Germany is complete

In [ ]:
set(lc['GERMANY']) - set(lc['ENGLISH'])

In [ ]:
set(lc['GERMANY']) - set(lc['ENGLISH'])

In [ ]:
len(books)

In [ ]:
len(chaps)

In [ ]:
8217 - 260


In [ ]:
[ f for f, s in iter_fixed(data, book='MRK', lang='GERMANY', chap='MRK.16')]

In [ ]:
#del lc['GUARANI']
#del lc['CANELA']
del[o[0]]

In [ ]:
s = set(lc['GERMANY'])
for l, v in lc.items():
    v = set(v)
    s = s & v
    if not s or not v:
        print(l)
    

In [ ]:
len(s) / len(lc['GERMANY'])

In [ ]:
o = sorted([ l for l in lc.keys()], key=lambda x: len(lc[x]))

In [ ]:
o[0]

In [ ]:
del lc[o[0]]

In [ ]:
from math import isnan
for e, v in iter_fixed(data, lang=o[0]):
    if e.vers == 0 and not(isnan(v)):
        print(e, v)

In [ ]:
r = set()
for l in o:
    r.add(l)
    s = set(lc['GERMANY'])
    for l1, v in lc.items():
        if l1 in r: continue 
        v = set(v)
        s = s & v
    print(len(r), len(lc.keys()) - len(r), len(s), len(s)/len(lc['GERMANY']))

In [ ]:
len(o[31:])